# Latent Variable Models and Variational Bayes


### Preliminaries

- Goal 
  - Introduction to latent variable models and variational inference    
- Materials
  - Mandatory
    - These lecture notes
  - Optional 
    - Bishop pp. 55-57 for Jensen's inequality
  - references
    - Blei et al. (2017), [Variational Inference: A Review for Statisticians](https://www.tandfonline.com/doi/full/10.1080/01621459.2017.1285773) 
    

### Illustrative Example

- You're now asked to build a density model for a data set ([Old Faithful](https://en.wikipedia.org/wiki/Old_Faithful), Bishop pg. 681) that clearly is not distributed as a single Gaussian:

<img src="./figures/fig-Bishop-A5-Old-Faithfull.png" width="350">

###  Unobserved Classes

Consider again a set of observed data $D=\{x_1,\dotsc,x_N\}$

- This time we suspect that there are _unobserved_ class labels that would help explain (or predict) the data, e.g.,
  - the observed data are the color of living things; the unobserved classes are animals and plants.
  - observed are wheel sizes; unobserved categories are trucks and personal cars.
  - observed is an audio signal; unobserved classes include speech, music, traffic noise, etc.
 

   
- Classification problems with unobserved classes are called **Clustering** problems. The learning algorithm needs to **discover the classes from the observed data**.

### The Gaussian Mixture Model

- Let $D=\{x_n\}$ be a set of observations. We associate a one-hot coded hidden class label $z_n$ with each observation:
$$
z_{nk} = \begin{cases} 1 & \text{if } x_n \in \mathcal{C}_k \text{ (the k-th class)}\\
                       0 & \text{otherwise} \end{cases}
$$

- We consider the same model as for the generative classification model:
$$\begin{align*}
p(x_n | z_{nk}=1) &= \mathcal{N}\left( x_n | \mu_k, \Sigma_k\right)\\
p(z_{nk}=1) &= \pi_k
\end{align*}$$
which can be summarized with the one-hot coded selection variables $z_{nk}$ as
$$\begin{align*}
p(x_n,z_n) &= p(x_n | z_n) p(z_n) = \prod_{k=1}^K \left( \pi_k \cdot \mathcal{N}\left( x_n | \mu_k, \Sigma_k\right) \right)^{z_{nk}} 
\end{align*}$$
- The model for an _observed_ data point $x_n$ is now given by (<font color=green>exercise B-9.3</font>)
$$\begin{align*}{}
p(x_n) &= \sum_{z_n} p(x_n,z_n)  \\
  &= \sum_{k=1}^K \pi_k \cdot \mathcal{N}\left( x_n | \mu_k, \Sigma_k \right) \tag{B-9.12}
\end{align*}$$
- This model is known as a <font color=red>Gaussian Mixture Model</font> (GMM).


###  GMM is a very flexible model

- GMMs are very popular models. They have decent computational properties and are **universal approximators of densities** (as long as there are enough Gaussians of course)

<img src="./figures/fig-ZoubinG-GMM-universal-approximation.png" width="600">

- (In the above figure, the Gaussian components are shown in <font color=red>red</font> and the pdf of the mixture models in <font color=blue>blue</font>).

###  The Kullback-Leibler Divergence

- In order to prove that the EM algorithm works, we will need [Gibbs inequality](https://en.wikipedia.org/wiki/Gibbs%27_inequality), which is a famous theorem in information theory.

- Definition: the **Kullback-Leibler divergence** (a.k.a. **relative entropy**) is a distance measure between two distributions $q$ and $p$ and is defined as

$$
D_{\text{KL}}(q \parallel p) \triangleq \sum_z q(z) \log \frac{q(z)}{p(z)}
$$

### The Free Energy Functional

- Consider a generative model specified by $$p(x,z) = p(x|z)p(z)\,,$$ where $x$ and $z$ represent the observed and unobserved variables, respectively.
- Assume that $x$ has been observed and we are interested in performing Bayesian inference, i.e., we want to compute the posterior $p(z|x)$ for the latent variables and the evidence $p(x)$.
- Consider the following loss functional (the **variational free energy** or shortly **free energy** (FE))
$$\begin{align}
F[q] &\triangleq \sum_z q(z) \log \frac{q(z)}{p(x,z)} \tag{B-10.3}
\end{align}$$
  - (As an aside), note that Bishop introduces an _Evidence Lower BOund_ (in modern literature abbreviated as ELBO) $\mathcal{L}(z)$ that equals the _negative_ FE. We prefer to discuss variational inference in terms of a free energy (but it is the same story as he discusses with the ELBO).  

- <a id='fe-decompositions'></a>Note that the FE can be decomposed as (link to [exercise](#exercises))
$$\begin{align}
F[q] &= \underbrace{-\sum_z q(z) \log p(x,z)}_{\text{energy}} - \underbrace{\sum_z q(z) \log \frac{1}{q(z)}}_{\text{entropy}} \tag{EE}\\
  &= \underbrace{\sum_z q(z) \log \frac{q(z)}{p(z|x)}}_{\text{divergence}} - \underbrace{\log p(x)}_{\text{log-evidence}}  \tag{DE}\\
  &=  \underbrace{\sum_z q(z)\log\frac{q(z)}{p(z)}}_{\text{complexity}} - \underbrace{\sum_z q(z) \log p(x|z)}_{\text{accuracy}}  \tag{IC}
\end{align}$$
  - These decompositions are very insightful and we will label them respectively as _energy-entropy_ (EE), _divergence-evidence_ (DE), and _complexity-accuracy_ (CA) decompositions. 
- The RHS of the DE decomposition does not depend on $q$. Hence, the global minimum $$q^* \triangleq \arg\max_q F[q]$$ is obtained for $q^*(z) = p(z|x)$, which is the Bayesian posterior.
- Furthermore, the minimal free energy $F^* \triangleq F[q^*] = -\log p(x)$ equals minus-log-evidence.  


$\Rightarrow$ <font color=red>It follows that Bayesian inference (computation of posterior and evidence) can be executed by FE minimization.</font>



###  Solving FE Minimization through Mean-field Factorization

- Note that the FE is a _functional_ , i.e., the FE is a function ($F$) of a function ($q$). We are looking for a _function_ $q^*(z)$ that minimizes the FE. 
- The mathematics of minimizing funstionals is described by _variational calculus_ , see Bishop app.D and Lanczos. (Optional reading).
- Generally speaking, it is not possible to solve the variational FE minimization problem without adding some constraints on $q(z)$.
- We will discuss three important cases of setting constraints on $q(z)$
  1. mean-field factorization
  2. parameterization
  3. the EM algorithm
  
- One important constraint is the so-called _mean-field_ constraint, given by
$$
q(z) = \prod_{i=1}^m q_i(z_i)\,, \tag{B-10.5}
$$
i.e., the posterior factorizes into $m$ _independent_ factors $q_i(z_i)$.

- Given the mean-field constraint, it is possible to derive an expression for the optimal solutions $q_j^*(z_j)$, for $j=1,\ldots,m$, which is given by (see Bishop, p.466, eq.10.9)
$$\begin{align} 
\log q_j^*(z_j) &\propto \mathrm{E}_{q_{-j}^*}\left[ \log p(x,z) \right] \tag{B-10.9}\\
  &=\sum_{z_{-j}} q_{-j}^*(z_{-j}) \log p(x,z) \notag
\end{align}$$
where we defined $q_{-j}^*(z_{-j}) \triangleq q_1^*(z_1)q_2^*(z_2)\cdots q_{j-1}^*(z_{j-1})q_{j+1}^*(z_{j+1})\cdots q_m^*(z_m)$.
  - <font color=green>Proof (from Blei (2017)): We first rewrite the energy-entropy decomposition of the FE as a function of $q_j(z_j)$ only: 
  $$ F[q_j] = \mathbb{E}_{q_{j}}\left[ \mathbb{E}_{q_{-j}}\left[ \log p(x,z_j,z_{-j})\right]\right] - \mathbb{E}_{q_j}\left[ \log q_j(z_j)\right] + \mathtt{const.}\,,$$
  where the constant holds all terms that do not depend on $q_j(z_j)$. This expression can be written as 
  $$ F[q_j] = \sum_{z_j} q_j(z_j) \log \frac{q_j(z_j)}{\exp\left( \mathbb{E}_{q_{-j}}\left[ \log p(x,z_j,z_{-j})\right]\right)}$$
  which is a KL-divergence that is minimized by Eq. B-10.9. </font> 
  
- This is not yet a full solution to the FE minimization task since the solution $q_j^*(z_j)$ depends on expectations that involve $q_{i\neq j}^*(z_{i \neq j})$, and each of the solutions $q_{i\neq j}^*(z_{i \neq j})$ depends on an expection that involves $q_j^*(z_j)$. 
- In practice, we solve this chicken-and-egg problem by an iterative approach: we first initialize all $q_j(z_j)$ (for $j=1,\ldots,m$) to an appropriate initial distribution and then cycle through the factors in turn by solving eq.10.9 and update $q_{-j}^*(z_{-j})$ with the latest estimates. (See Blei (2017), Algorithm 1, p864).  


### Example: FE Minimization by Mean-field Factorization

- Let's work out an FE minimization task for a simple Gaussian estimation problem. Bishop section 10.1.3
- <font color=red> TBD <font>


### FEM for the Gaussian Mixture Model

##### model specification


- We consider a Gaussian Mixture Model, specified by 
$$\begin{align*}
p(x,z|\theta) &= p(x|z,\mu,\Lambda)p(z|\pi) \\
  &= \prod_{n=1}^N \prod_{k=1}^K \pi_k^{z_{nk}}\cdot \mathcal{N}\left( x_n | \mu_k, \Lambda_k^{-1}\right)^{z_{nk}} \tag{B-10.37,38}
\end{align*}$$
with tuning parameters $\theta=\{\pi_k, \mu_k,\Lambda_k\}$. 
- Let us introduce some priors for the parameters. We factorize the prior as
$$
p(\theta) = p(\pi,\mu,\Lambda) = p(\pi) p(\mu|\Lambda) p(\Lambda)
$$
with 
$$\begin{align*}
p(\pi) &= \mathrm{Dir}(\pi|\alpha_0) = C(\alpha_0) \prod_k \pi_k^{\alpha_0-1} \tag{B-10.39}\\
p(\mu|\Lambda) &= \prod_k \mathcal{N}\left(\mu_k | m_0, \left( \beta_0 \Lambda_k\right)^{-1} \right) \tag{B-10.40}\\
p(\Lambda) &= \prod_k \mathcal{W}\left( \Lambda_k | W_0, \nu_0 \right) \tag{B-10.40}
\end{align*}$$

- The full generative model is now specified by
$$
p(x,z,\pi,\mu,\Lambda) = p(x|z,\mu,\Lambda) p(z|\pi) p(\pi) p(\mu|\Lambda) p(\Lambda) \tag{B-10.41}
$$
with hyperparameters $\{ \alpha_0, m_0, \beta_0, W_0, \nu_0\}$.

##### inference

- Assume that we have observed $D = \left\{x_1, x_2, \ldots, x_N\right\}$ and are interested to infer the posterior distribution for the tuning parameters: 
$$
p(\theta|D) = p(\pi,\mu,\Lambda|D)
$$

- The (perfect) Bayesian solution is 
$$
p(\theta|D) = \frac{p(x=D,\theta)}{p(D)}\frac{\sum_z p(x=D,z,\pi,\mu,\Lambda)}{\sum_z \sum_{\pi} \iint p(x=D,z,\pi,\mu,\Lambda) \,\mathrm{d}\mu\mathrm{d}\Lambda}
$$
but this is intractable (See Blei (2017), p861, eqs. 8 and 9).

- Bishop shows that the the mean-field optimal equations (Eq. B-10.9) are analytically solvable for the GMM as specified above, with the following solution: 


### Code Example:

### FE Minimization by the Expectation-Maximization (EM) Algorithm

- The EM algorithm is a special case of FE minimization that focusses on Maximum-Likelihood estimation for models with latent variables. 
- Consider a model $$p(x,z,\theta)$$ with observations $x = \{x_n\}$, latent variables $z=\{z_n\}$ and parameters $\theta$.

- We can write the following FE functional for this model:
$$\begin{align}
F[q] =  \sum_z \sum_\theta q(z) q(\theta) \log \frac{q(z) q(\theta)}{p(x,z,\theta)} 
\end{align}$$

- The EM algorithm makes the following simplifying assumptions:
  1. The prior for the parameters is uninformative (uniform). This implies that 
  $$p(x,z,\theta) = p(x,z|\theta) p(\theta) \propto p(x,z|\theta)$$
  2. The posterior for the parameters is a delta function:
  $$q(\theta) = \delta(\theta - \hat{\theta})$$
  
- Basically, these two assumptions turn FE minimization into maximum likelihood estimation for the parameters $\theta$ and the FE simplifies to 
$$\begin{align}
F[q,\theta] =  \sum_z q(z) \log \frac{q(z)}{p(x,z|\theta)} 
\end{align}$$

- The EM algorithm minimizes this FE by iterating (iteration counter: $i$) over 

<font color=red>$$\begin{align*}
q(z) &= p(z|x,\theta^{(i-1)}) \tag{the E-step}\\
\theta^{(i)} &= \arg\max_\theta \sum_z q(z) \log p(x,z|\theta^{(i-1)}) \tag{the M-step}
\end{align*}$$</font>

- These choices are optimal for the given FE functional. In order to see this, consider the two decompositions
$$\begin{align}
F[q,\theta] &= \underbrace{-\sum_z q(z) \log p(x,z|\theta)}_{\text{energy}} - \underbrace{\sum_z q(z) \log \frac{1}{q(z)}}_{\text{entropy}} \tag{EE}\\
  &= \underbrace{\sum_z q(z) \log \frac{q(z)}{p(z|x,\theta)}}_{\text{divergence}} - \underbrace{\log p(x|\theta)}_{\text{log-likelihood}}  \tag{DE}
\end{align}$$

- The DE decomposition shows that the FE is minimized for the choice $q(z) := p(z|x,\theta)$. Also, for this choice, the FE equals the (negative) log-evidence. 

- The EE decomposition shows that the FE is minimized wrt $\theta$ by minimizing the energy term (which is indeed the choice in the M-step of the EM algorithm).
  - (In the EM literature, this term is called the _expected complete-data log-likelihood_.)

- In order to execute the EM algorithm, it is assumed that we can analytically execute the E- and M-steps. For a large set of models (including models whose distributions belong to the exponential family of distributions), this is indeed the case and hence the large popularity of the EM algorithm. 

- The EM algorihm imposes rather severe assumptions on the FE. Over the past few years, the rise of Probabilistic Programming languages has dramatically increased the range of models for which the parameters can by estimated autmatically by (approximate) Bayesian inference, so the popularity of EM is slowly waning. (More on this in the Probabilistic Programming lessons). 

- Bishop (2006) works out EM for the GMM in section 9.2.

-  Theorem: **Gibbs Inequality** ([proof](https://en.wikipedia.org/wiki/Gibbs%27_inequality#Proof) uses Jensen inquality):    
$$\boxed{ D_{\text{KL}}(q \parallel p) \geq 0 }$$
with equality only iff $p=q$.

- Note that the KL divergence is an asymmetric distance measure, i.e. in general 

$$D_{\text{KL}}(q \parallel p) \neq D_{\text{KL}}(p \parallel q)$$

###  EM by maximizing a Lower Bound on the Log-Likelihood

- Consider a model for observations $x$, hidden variables $z$ and tuning parameters $\theta$. Note that, for **any** distribution $q(z)$, we can expand the log-likelihood as follows: 

$$\begin{align*}
\mathrm{L}(\theta) &\triangleq \log p(x|\theta)  \\
  &= \sum_z q(z) \log p(x|\theta) \\
  &= \sum_z q(z) \left( \log p(x|\theta) - \log \frac{q(z)}{p(z|x,\theta)}\right) +  \sum_z q(z) \log \frac{q(z)}{p(z|x,\theta)} \\
  &= \sum_z q(z) \log \frac{p(x,z|\theta)}{q(z)} + \underbrace{D_{\text{KL}}\left( q(z) \parallel p(z|x,\theta) 
\right)}_{\text{Kullback-Leibler div.}} \tag{1} \\
  &\geq \sum_z q(z) \log \frac{p(x,z|\theta)}{q(z)} \quad \text{(use Gibbs inequality)}  \\
  &= \underbrace{\sum_z q(z) \log p(x,z|\theta)}_{\text{expected complete-data log-likelihood}} + \underbrace{\mathcal{H}\left[ q\right]}_{\text{entropy of }q}  \\
&\triangleq \mathrm{LB}(q,\theta) 
\end{align*}$$

- Hence, $\mathrm{LB}(q,\theta)$ is a **lower bound** on the log-likelihood $\log p(x|\theta)$. 

- Technically, the Expectation-Maximization (EM) algorithm is defined by coordinate ascent on $\mathrm{LB}(q,\theta)$:

$$\begin{align*}
  &\textrm{ } \\
  &\textrm{Initialize }: \theta^{(0)}\\
  &\textrm{for }m = 1,2,\ldots \textrm{until convergence}\\
    &\quad q^{(m+1)} = \arg\max_q \mathrm{LB}(q,\theta^{(m)}) &\quad \text{% update responsibilities} \\
    &\quad \theta^{(m+1)} = \arg\max_\theta \mathrm{LB}(q^{(m+1)},\theta) &\quad \text{% re-estimate parameters}
\end{align*}$$

- Since $\mathrm{LB}(q,\theta) \leq \mathrm{L}(\theta)$ (for all choices of $q(z)$), maximizing the lower-bound $\mathrm{LB}$ will also maximize the log-likelihood wrt $\theta$. The _reason_ to maximize $\mathrm{LB}$ rather than log-likelihood $\mathrm{L}$ directly is that $\arg\max \mathrm{LB}$ often leads to easier expressions. E.g., see this illustrative figure (Bishop p.453):

<img src="./figures/Bishop-Figure914.png" width="400px">

### EM Details

##### Maximizing $\mathrm{LB}$ w.r.t. $q$ 

- Note that
$$
\mathrm{LB}(q,\theta) =  \mathrm{L}(\theta)  - D_{\text{KL}}\left( q(z) \parallel p(z|x,\theta) \right)
$$
and consequenty, maximizing $\mathrm{LB}$ over $q$ leads to minimization of the KL-divergence. Specifically, it follows from Gibbs inequality that
$$
 q^{(m+1)}(z) := p(z|x,\theta^{(m)})
$$

##### Maximizing $\mathrm{LB}$ w.r.t. $\theta$

- It also follows from (the last line of) the multi-line derivation above that maximizing $\mathrm{LB}$ w.r.t. $\theta$ amounts to maximization of the _expected complete-data log-likelihood_ (where the complete data set is defined as $\{(x_i,z_i)\}_{i=1}^N$). Hence, the EM algorithm comprises iterations of
$$
\boxed{\textbf{EM}:\, \theta^{(m+1)} := \underbrace{\arg\max_\theta}_{\text{M-step}} \underbrace{\sum_z  \overbrace{p(z|x,\theta^{(m)})}^{=q^{(m+1)}(z)} \log p(x,z|\theta)}_{\text{E-step}} }
$$

<!---
- Compare this to regular log-likelihood maximization:
$$
\boxed{\textbf{ML}:\, \hat \theta:= \arg\max_\theta \log p(x|\theta)}
$$
--->


### <span style="color:green">Homework exercise: EM for GMM Revisited</span>

##### E-step
- Write down the GMM generative model
- The complete-data set is $D_c=\{x_1,z_1,x_2,z_2,\ldots,x_n,z_n\}$. Write down the _complete-data_ likelihood $p(D_c|\theta)$
- Write down the complete-data _log_-likelihood $\log p(D_c|\theta)$
- Write down the _expected_ complete-data log-likelihood $\mathrm{E}_Z\left[ \log p(D_c|\theta) \right]$


##### M-step
- Maximize $\mathrm{E}_Z\left[ \log p(D_c|\theta) \right]$ w.r.t. $\theta=\{\pi,\mu,\Sigma\}$

- Verify that your solution is the same as the 'intuitive' solution of the previous lesson. 

###  <span style="color:green">Homework Exercise: EM for Three Coins problem</span>

- You have three coins in your pocket. For each coin, outcomes $\in \{\mathrm{H},\mathrm{T}\}$.
$$
p(\mathrm{H}) = \begin{cases} \lambda & \text{for coin }0 \\
 \rho & \text{for coin }1 \\
 \theta & \text{for coin }2 \end{cases}
$$

    
-  **Scenario**. Toss coin $0$. If Head comes up, toss three times with coin $1$; otherwise, toss three times with coin $2$.

- The observed sequences **after** each toss with coin $0$ were $\langle \mathrm{HHH}\rangle$, $\langle \mathrm{HTH}\rangle$, $\langle \mathrm{HHT}\rangle$, and $\langle\mathrm{HTT}\rangle$

- **Task**. Use EM to estimate most the likely values for $\lambda$, $\rho$ and $\theta$





###  Report Card on EM

-  EM is a general procedure for learning in the presence of unobserved variables.

-  In a sense, it is a **family of algorithms**. The update rules you will derive depend on the probability model assumed.

-  (Good!) **No tuning parameters** such a learning rate, unlike gradient descent-type algorithms

-  (Bad). EM is an iterative procedure that is very sensitive to initial
conditions! EM converges to a **local optimum**.

-  Start from trash $\rightarrow$ end up with trash. Hence, we need a good and fast initialization procedure (often used: K-Means, see Bishop p.424)

-  Also used to train HMMs, etc.

### <span style="color:red">(OPTIONAL SLIDE)</span> The Free-Energy Principle

- The negative lower-bound $\mathrm{F}(q,\theta) \triangleq -\mathrm{LB}(q,\theta)$ also appears in various scientific disciplines. In statistical physics and variational calculus, $F$ is known as the **free energy** functional. Hence, the EM algorithm is a special case of **free energy minimization**.  

- It follows from Eq.1 above that

$$\begin{align*}
\mathrm{F}(q,\theta) &\triangleq \sum_z q(z) \log \frac{q(z)}{p(x,z|\theta)} \\
  &= \underbrace{- \mathrm{L}(\theta)}_{-\text{log-likelihood}} + \underbrace{D_{\text{KL}}\left( q(z) \parallel p(z|x,\theta) 
\right)}_{\text{divergence}}
\end{align*}$$ 

- The **Free-Energy Principle** (FEP) is an influential [neuroscientific theory](https://en.wikipedia.org/wiki/Free_energy_principle) that claims that information processing in the brain is also an example of free-energy minimization, see [Friston, 2009](./files/Friston-2009-The-free-energy-principle-a-rough-guide-to-the-brain.pdf).

- According to FEP, the brain contains a generative model $p(x,z,a,\theta)$ for its environment. Here, $x$ are the sensory signals (observations); $z$ corresponds to (hidden) environmental causes of the sensorium; $a$ represents our actions (control signals for movement) and $\theta$ describes the fixed 'physical rules' of the world.

### <span style="color:red">(OPTIONAL SLIDE)</span> The Free-Energy Principle, cont'd

- Solely through free-energy minimization, the brain infers an approximate posterior $q(z,a,\theta)$, thus inferring our **perception**, **actions**, and **learning** equations.

- Free-energy can be interpreted as (a generalized notion of sum of) prediction errors. Free-energy minimization aims to minimize prediction errors through perception, actions, and learning. (The next picture  is from a [2012 tutorial presentation](http://slideplayer.com/slide/9925565/) by Karl Friston) 

<img src="./figures/friston-fep.jpg" width="500px">

### <span style="color:red">(OPTIONAL SLIDE)</span> The Free-Energy Principle, cont'd

- $\Rightarrow$ The brain is "nothing but" an approximate Bayesian agent that tries to build a model for its world. Actions (behavior) are selected to fulfill prior expectations about the world. 

- In our [BIASlab research team](http://biaslab.org) in the Signal Processing Systems group (FLUX floor 7), we work on developing (approximately Bayesian) **artificial** intelligent agents that learn purposeful behavior through interactions with their environments, inspired by the free-energy principle. Applications include
  - robotics
  - self-learning of new signal processing algorithms, e.g., for hearing aids
  - self-learning how to drive



- Let me know (email bert.de.vries@tue.nl) if you're interested in developing machine learning applications that are directly inspired by how the brain computes. We often have open traineeships or MSc thesis projects available. 

### <a id='exercises'></a>Exercises

1. Given the free energy , proof the [EE, DE and AC decompositions](#fe-decompositions) 

In [1]:
open("../../styles/aipstyle.html") do f
    display("text/html", read(f, String))
end

<!--
This HTML file contains custom styles and some javascript.
Include it a Jupyter notebook for improved rendering.
-->

<!-- Fonts -->
 
 
 
 
 

<!-- Custom style -->


<!-- MathJax styling -->